In [1]:
#############################################################
# Compile `main.cu` to `file.exe`
# Commented out because we don't want to run it all the time
#############################################################
!nvcc -w functions.cu main.cu -o file

functions.cu
tmpxft_000084ac_00000000-10_functions.cudafe1.cpp
main.cu
tmpxft_000084ac_00000000-14_main.cudafe1.cpp
   Creating library file.lib and object file.exp


In [3]:
#############################################################
# Load python dependencies
#############################################################
import numpy as np
import matplotlib as plt

# TODO: import utils functions
# Read into detuning
# Calculate Td theory and test
# Subplots


# Generate detuning
todo: distribution functions
read into detuning.dat

In [2]:
#############################################################
# Generate and write detuning data to file
#############################################################
# Write 1D numpy array to file
detuning_values_1d = np.array([-150, -50, 0, 50, 150])
print("Detuning data:", detuning_values_1d)
np.savetxt("Detuning.dat", detuning_values_1d, delimiter="\t")

# Example: write 2D numpy array to file
# values_2d = np.array([
#     [10.0, 20.0, 40.0],
#     [10.0, 20.0, 50.0],
#     [10.0, 20.0, 60.0]
# ])
# np.savetxt("values_2d.dat", values_2d, delimiter="\t")

Detuning data: [-150  -50    0   50  150]


# Run simulation

In [2]:

#############################################################
# Run multiple jobs
#############################################################
# List of arguments for each run
runs = [
    {"num_ens": 5, "n_total": 100000, "theta_0": 1.0, "phi_0": 0.0, "coup_a_0": 1.6, "gamma_a_0": 0.0, "chi_a_0": 0.0, "kappa_c_0": 160.0, "t_max": 0.5, "t_num": 60000, "handle": "60000"},
    {"num_ens": 5, "n_total": 100000, "theta_0": 1.0, "phi_0": 0.0, "coup_a_0": 1.6, "gamma_a_0": 0.0, "chi_a_0": 0.0, "kappa_c_0": 160.0, "t_max": 0.5, "t_num": 70000, "handle": "70000"},
    {"num_ens": 5, "n_total": 100000, "theta_0": 1.0, "phi_0": 0.0, "coup_a_0": 1.6, "gamma_a_0": 0.0, "chi_a_0": 0.0, "kappa_c_0": 160.0, "t_max": 0.5, "t_num": 80000, "handle": "80000"},
    {"num_ens": 5, "n_total": 100000, "theta_0": 1.0, "phi_0": 0.0, "coup_a_0": 1.6, "gamma_a_0": 0.0, "chi_a_0": 0.0, "kappa_c_0": 160.0, "t_max": 0.5, "t_num": 90000, "handle": "90000"},
    {"num_ens": 5, "n_total": 100000, "theta_0": 1.0, "phi_0": 0.0, "coup_a_0": 1.6, "gamma_a_0": 0.0, "chi_a_0": 0.0, "kappa_c_0": 160.0, "t_max": 0.5, "t_num": 100000, "handle": "100000"},
    {"num_ens": 5, "n_total": 100000, "theta_0": 1.0, "phi_0": 0.0, "coup_a_0": 1.6, "gamma_a_0": 0.0, "chi_a_0": 0.0, "kappa_c_0": 160.0, "t_max": 0.5, "t_num": 110000, "handle": "110000"}
]

for arg in runs:
    # Prepare run arguments
    num_ens = arg['num_ens']
    n_total = arg['n_total']
    theta_0 = arg['theta_0']
    phi_0 = arg['phi_0']
    coup_a_0 = arg['coup_a_0']
    gamma_a_0 = arg['gamma_a_0']
    chi_a_0 = arg['chi_a_0']
    kappa_c_0 = arg['kappa_c_0']
    t_max = arg['t_max']
    t_num = arg['t_num']
    handle = arg['handle']
    
    # Run simulation with muted stdout stderr, 1st line for WIN, 2nd for UNIX. To view progress, use terminal instead
    # !file.exe {num_ens} {n_total} {theta_0} {phi_0} {coup_a_0} {gamma_a_0} {chi_a_0} {kappa_c_0} {t_max} {t_num} {handle}  >nul 2>&1
    # !file.exe {num_ens} {n_total} {theta_0} {phi_0} {coup_a_0} {gamma_a_0} {chi_a_0} {kappa_c_0} {t_max} {t_num} {handle}  > /dev/null 2>&1
    
    # Construct the result data filenames based on the `handle`
    result_sz_filename = f"Result_Sz_{handle}.dat"
    result_coherences_filename = f"Result_coherences_real_{handle}.dat"
    result_photon_filename = f"Result_photon_{handle}.dat"
    result_time_filename = f"Result_time_{handle}.dat"
    
    result_sz = np.loadtxt(result_sz_filename, dtype=np.longdouble)
    result_time = np.loadtxt(result_time_filename, dtype=np.longdouble)
    detuning = np.loadtxt("Detuning.dat", dtype=np.longdouble)
    
    print(detuning)
    print(result_time)
    
    res = np.rot90(result_sz)
    print(result_sz)
    print(res)
    
    # # Load result files and plot it
    # # TODO: Not sure how to plot the result data
    # for filename in [
    #         result_sz_filename, 
    #         result_coherences_filename, 
    #         result_photon_filename, 
    #         result_time_filename
    #     ]:
    #     result = np.loadtxt(filename, dtype=np.longdouble)
    #     print(filename)
    #     print(result)
    #     plt.plot(result)
    #     plt.show()
        
        

NameError: name 'np' is not defined

# Analyze & Plot
todo:  import util functions return predict Td
write block to plot with Td and found Td

In [ ]:
import matplotlib.pyplot as plt

# Compute heat map (for pop. inversion or light emission)
def plot_heat(x, y, z, z_min, z_max):
    # x,y: x and y axis variables
    # z: dependent variable, dimension dim(x)*dim(y)
    # z_min, z_max: range of z to plot
    c = plt.pcolormesh(x, y, z, cmap = 'CMRmap', vmin = z_min, vmax = z_max)
    plt.colorbar(c)
    plt.title('$<\sigma_z>$',fontsize=18)
    plt.ylabel('Detuning',fontsize=18)
    plt.xlabel('Time$(\mu s)$',fontsize=18)

In [1]:
runs[1]

NameError: name 'runs' is not defined